In [1]:
from __future__ import division

from pulp import *
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.mlab as mlab
import pandas as pd
import csv
import datetime
import sqlite3
import math
import glob

import seaborn as sns
%matplotlib inline

from pandas import *
from scipy import optimize
from scipy import stats
from datetime import date, timedelta, datetime
import timeit
from collections import OrderedDict

# from IPython.display import clear_output
# from IPython.display import HTML
from matplotlib.ticker import MultipleLocator

# Formulate the Objective Function

In the object function, we include the following cost function and constrains: 

Query each distinct event, understand the start and end time slots. 

if event has slack:

    check hasSlack to True in sessiondata
    
    calculate total energy cost (EC)
    
    write it to CurrentEC field in sessiondata
    
    optimize within the start and end period
    
    write it to OptimizedEC field in sessiondata
    
    write the new schedule EVSE_ID, START_TIME, END_TIME, AVG_POWER, SUBLAP, EVSE_ZIP into CostOptimizedMinuteData
    
else

    calculate the total Energy Cost
    
    write it to CurrentEC and OptimizedEC field in sessiondata
    
finally
    check hasECOptimized to True in sessiondata


## Read all drivers' Session data to DataFrame

Query each event on the day, understand the start and end time slots.
if event has slack:
    check hasSlack to True in data
    
    calculate total energy charges
    
    write it to CurrentEC field in data
    
    optimize within the start and end period
    
    write it to OptimizedEC field in data
    
    write the new schedule into data

else
    
    calculate the total energy charges
    
    write it to CurrentEC and OptimizedEC field in data
    
finally
    check has hasECOptimized to True in data

In [18]:
def query_EVSE_eventids(startdate, enddate):
    # Query each event, for each event run the optimization based on corresponding energy cost to minimize 
    # the sum of energy cost every 15 minutes.
    print('Start to query EV power at "Alcopark"')

    # Get distinct event_ids for 
    conn = sqlite3.connect("sessionData2016.db")
    cursor = conn.cursor()
    query=("SELECT DISTINCT(EVENT_ID) FROM sessionData2016")
    query_cond1=("where ENERGY>0 and ENERGY!='' AND")
    query_cond2=("STATION LIKE '%ALCO%' AND") # ALCOPARK
    query_cond3=("(strftime(SESSION_START_TIME) between ") # MONTH IN JAN
    cur = cursor.execute(query+" "+query_cond1+" "+query_cond2+" "+query_cond3+"'"+startdate+"'"+' AND '+"'"+enddate+"'"+')')
    rows = cur.fetchall()
    event_ids=[]
    for row in rows: event_ids.append(row[0])
    event_ids=np.array(event_ids)
    conn.close()
    
    # query EVSE data from SQL
    EVSE_Power = pd.DataFrame()
    index = pd.date_range(startdate, enddate, freq='15min')

    EVSE_Power = pd.DataFrame(index=index)

    tcts=len(event_ids)
    for event_id in event_ids:
#         print('This is the current EVENT_ID {}'.format(event_id))
#         if tcts % 30 ==0: clear_output()
#         start = timeit.default_timer()   
        conn = sqlite3.connect("minData2016.db")
        cursor = conn.cursor()
        # FORM A QUERY TO GET RELEVANT DATA, MAKE SURE TO FLOOR THE STARTING OR ENDING POINT
        # THIS WUERY WILL RETURN INTERVAL_ENERGY IN KWH, THIS REALLY IS WHAT WE ARE INTERESTED IN FOR OPTIMIZATION 
        # WE WILL NEED THE TIME FIELD TO GET THE CORRESPONDING VALUES OF ENERGY COST
        # THE REST WILL BE USED WHEN POPULATING THE MINDATA DATABASE
        query=("SELECT INTERVAL_START_DATE AS START,INTERVAL_END_DATE AS END,AVERAGE_POWER,ENERGY FROM minData2016 WHERE ENERGY!='' AND EVENT_ID=")
        query_cond=str(event_id)
        cur = cursor.execute(query+query_cond)
        df = pd.DataFrame(cur.fetchall())
        df.columns=['start','end','power','cumulative_energy']
        df['StartDate'] = df['start'].apply(lambda x: datetime.strptime(x,'%m/%d/%y %H:%M'))
        df.index = df['StartDate']
        try:
            df = df.resample('15min').bfill()
            df = df.rename(columns = {'power':event_id})
            EVSE_Power = pd.concat([EVSE_Power,pd.DataFrame(df[event_id])], axis = 1)
        except ValueError:
            print('Oops! That was no valid Event ID: ' + str(event_id))

    return EVSE_Power


In [ ]:
def query_EVSE_eventids(startdate, enddate):
    # Query each event, for each event run the optimization based on corresponding energy cost to minimize 
    # the sum of energy cost every 15 minutes.
    print('Start to query EV power at "Alcopark"')

    # Get distinct event_ids for 
    conn = sqlite3.connect("sessionData2016.db")
    cursor = conn.cursor()
    query=("SELECT DISTINCT(EVENT_ID) FROM sessionData2016")
    query_cond1=("where ENERGY>0 and ENERGY!='' AND")
    query_cond2=("STATION LIKE '%ALCO%'")
#     query_cond2=("STATION LIKE '%ALCO%' AND") # ALCOPARK

    # PARKING AT NIGHT, FLEET VEHICLES
#     query_cond3=("(((CAST(strftime('%j',SESSION_END_TIME) as INT) - CAST(strftime('%j',SESSION_START_TIME) as INT)) > 0) AND")
#     query_cond4=("(strftime('%Y-%m-%d',SESSION_START_TIME) between ") # MONTH IN JAN
#     cur = cursor.execute(query+" "+query_cond1+" "+query_cond2+" "+query_cond4+"'"+startdate+"'"+' AND '+"'"+enddate+"'"+')')
    cur = cursor.execute(query+" "+query_cond1+" "+query_cond2)
    rows = cur.fetchall()
    event_ids=[]
    for row in rows: event_ids.append(row[0])
    event_ids=np.array(event_ids)
    conn.close()
    
    # query EVSE data from SQL
    EVSE_Power = pd.DataFrame()
    index = pd.date_range(startdate, enddate, freq='15min')

    EVSE_Power = pd.DataFrame(index=index)

    tcts=len(event_ids)
    for event_id in event_ids:
#         print('This is the current EVENT_ID {}'.format(event_id))
#         if tcts % 30 ==0: clear_output()
#         start = timeit.default_timer()   
        conn = sqlite3.connect("minData2016.db")
        cursor = conn.cursor()
        # FORM A QUERY TO GET RELEVANT DATA, MAKE SURE TO FLOOR THE STARTING OR ENDING POINT
        # THIS WUERY WILL RETURN INTERVAL_ENERGY IN KWH, THIS REALLY IS WHAT WE ARE INTERESTED IN FOR OPTIMIZATION 
        # WE WILL NEED THE TIME FIELD TO GET THE CORRESPONDING VALUES OF ENERGY COST
        # THE REST WILL BE USED WHEN POPULATING THE MINDATA DATABASE
        query=("SELECT INTERVAL_START_DATE AS START,INTERVAL_END_DATE AS END,AVERAGE_POWER,ENERGY FROM minData2016 WHERE ENERGY!='' AND EVENT_ID=")
        query_cond=str(event_id)
        cur = cursor.execute(query+query_cond)
        df = pd.DataFrame(cur.fetchall())
        df.columns=['start','end','power','cumulative_energy']
        df['StartDate'] = df['start'].apply(lambda x: datetime.strptime(x,'%m/%d/%y %H:%M'))
        df.index = df['StartDate']
        try:
            df = df.resample('15min').bfill()
            df = df.rename(columns = {'power':event_id})
            EVSE_Power = pd.concat([EVSE_Power,pd.DataFrame(df[event_id])], axis = 1)
        except ValueError:
            print('Oops! That was no valid Event ID: ' + str(event_id))

    return EVSE_Power


In [19]:
# consider the holidays
holidays = pd.read_csv('holiday.csv', usecols=[0])
holidays['date'] = pd.to_datetime(pd.Series(holidays['date']), format='%m/%d/%y')
holidays['day'] = holidays.date.apply(lambda x: x.strftime('%Y-%m-%d'))

# utility tariff, E19-TOU
utility = []
utility = pd.read_csv('PGE_E19.csv', usecols=[1])

# meter charge
meter_charge = float("{0:.2f}".format(utility['Secondary'][0]))
# demand charge
summer_onpeak_demand = float("{0:.5f}".format(utility['Secondary'][1]))
summer_midpeak_demand = float("{0:.5f}".format(utility['Secondary'][2]))
summer_monthly_demand = float("{0:.5f}".format(utility['Secondary'][3]))
winter_midpeak_demand = float("{0:.5f}".format(utility['Secondary'][4]))
winter_monthly_demand = float("{0:.5f}".format(utility['Secondary'][5]))

# energy charge
summer_onpeak_energy = float("{0:.5f}".format(utility['Secondary'][6]))
summer_midpeak_energy = float("{0:.5f}".format(utility['Secondary'][7]))
summer_offpeak_energy = float("{0:.5f}".format(utility['Secondary'][8]))
winter_midpeak_energy = float("{0:.5f}".format(utility['Secondary'][9]))
winter_offpeak_energy = float("{0:.5f}".format(utility['Secondary'][10]))

# PDP charge and credits
cpp_charge = utility['Secondary'][11]
credits_onpeak_demand = utility['Secondary'][12]
credits_midpeak_demand = utility['Secondary'][13]
# credits_onpeak_energy = utility['Secondary'][14]
# credits_midpeak_energy = utility['Secondary'][15]

# Get cost value of energy charges for time
def getCostValueForTime(month,time,weekday):
    if (month in [1,2,3,4,11,12]) & (weekday > 0) & (weekday < 6):
        if (time >= '08:30') & (time < '21:30'):
            return winter_midpeak_energy
        elif (time >= '00:00') & (time < '08:30'):
            return winter_offpeak_energy
        elif time >= '21:30':
            return 0
    elif (month in [1,2,3,4,11,12]) & (weekday in [6,7]):
        return winter_offpeak_energy
    elif (month in [5,6,7,8,9,10]) & (weekday > 0) & (weekday < 6):
        if (time >= '00:00') & (time < '08:30'):
            return summer_offpeak_energy
        elif (time >= '08:30') & (time < '12:00'):
            return summer_midpeak_energy
        elif (time >= '12:00') & (time < '18:00'):
            return summer_onpeak_energy
        elif (time >= '18:00') & (time < '21:30'):
            return summer_midpeak_energy
        elif (time >= '21:30') & (time <= '23:45'):
            return summer_offpeak_energy
    elif (month in [5,6,7,8,9,10]) & (weekday in [6,7]):
        return summer_offpeak_energy
    else: 
        print "Error in month, time and weekday"

In [20]:
EVSE_Power = query_EVSE_eventids('6/1/16','7/1/16')

Start to query EV power at "Alcopark"


In [22]:
EVSE_Power.to_csv('results/FlexDrivers/NewEVStatus/Opt5/'+'2015-'+str(6)+'Original.csv')

In [21]:
EVSE_Power

,46350105,46349045,46349027,46349021,46346935,46342447,46342417,46342365,46342109,46341863,...,44874547,44871479,44871169,44869173,44864519,44859267,44858539,44855983,44852697,44844165
2016-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 01:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 01:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01 02:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
ts2015 = pd.read_csv('results/FlexDrivers/NewEVStatus/2015TimeStamp.csv')
ts2015.index = ts2015.ix[:,0].apply(lambda x: datetime.strptime(x,'%m/%d/%y %H:%M'))
ts2015.drop(ts2015.columns[[0]], axis=1, inplace=True)

ts2015['day'] = map(lambda x: x.strftime('%Y-%m-%d'),ts2015.index)
ts2015['month'] = map(lambda x: x.strftime('%m'),ts2015.index)
ts2015['hour'] = map(lambda x: x.strftime('%H'),ts2015.index)
ts2015['hour'] = ts2015.hour.astype(int)
ts2015['month'] = ts2015.month.astype(int)

In [23]:
def V2optimizeCost(DailyPower, eventid, onpeak, midpeak, peak):
    
    DayPower = DailyPower.fillna(0)

#     DailyPower['ts'] = range(96) # Add the row number of each charging power time step
    
    # Create matrix of slots
    X = LpVariable.dicts("ChargingStatus", [(i, j) for i in range(96) for j in range(96)], 0, 1, LpBinary)

    costs=[]
    for tim in DayPower.index:
        costs.append(getCostValueForTime(tim.month,tim.strftime('%H:%M'),tim.isoweekday()))
    cost={}
    for i in xrange(96):
        for j in xrange(96):
            cost[i,j]=costs[j]
   
    # Create model objective
    model = pulp.LpProblem("SmartCharging",pulp.LpMinimize)
    # Create model variables
    min_curMidPeak = LpVariable("Min MidPeak",0)
    min_curOnPeak = LpVariable("Min OnPeak",0)
    min_curPeak = LpVariable("Min Peak",0)
        
    tim = DayPower.index[0]
    if (tim.month in [1,2,3,4,11,12]) & (tim.isoweekday() in [1,2,3,4,5]):
        curMidPeak = [lpSum(lpSum(DayPower[eventid].tolist()[i]*X[i,j] for i in xrange(96)) + DayPower['power'].tolist()[j]) for j in xrange(34,86)]
        curPeak = [lpSum(lpSum(DayPower[eventid].tolist()[i]*X[i,j] for i in xrange(96)) + DayPower['power'].tolist()[j]) for j in xrange(96)]
        
        model += lpSum(DayPower[eventid].tolist()[i]*X[i,j]*cost[i,j] for i in xrange(96) for j in xrange(96)) + min_curPeak*17.33 + min_curMidPeak*0.13
        
        for item in curPeak:
            model += min_curPeak >= item
        model += min_curPeak >= peak
        for item in curMidPeak:
            model += min_curMidPeak >= item
        model += min_curOnPeak >= midpeak
        
    elif (tim.month in [1,2,3,4,11,12]) & (tim.isoweekday() in [6,7]):
        curPeak = [lpSum(lpSum(DayPower[eventid].tolist()[i]*X[i,j] for i in xrange(96)) + DailyPower['power'].tolist()[j]) for j in xrange(96)]
        
        model += lpSum(DayPower[eventid].tolist()[i]*X[i,j]*cost[i,j] for i in xrange(96) for j in xrange(96)) + min_curPeak*17.33
        
        for item in curPeak:
            model += min_curPeak >= item
        model += min_curPeak >= peak
        
    elif (tim.month in [5,6,7,8,9,10]) & (tim.isoweekday() in [1,2,3,4,5]):
        curMidPeak = [lpSum(lpSum(DayPower[eventid].tolist()[i]*X[i,j] for i in xrange(96)) + DailyPower['power'].tolist()[j]) for j in (range(34,48)+range(72,86))]
        curPeak = [lpSum(lpSum(DayPower[eventid].tolist()[i]*X[i,j] for i in xrange(96)) + DailyPower['power'].tolist()[j]) for j in xrange(96)]
        curOnPeak = [lpSum(lpSum(DayPower[eventid].tolist()[i]*X[i,j] for i in xrange(96)) + + DailyPower['power'].tolist()[j]) for j in xrange(48,72)]
        
        model += lpSum(DayPower[eventid].tolist()[i]*X[i,j]*cost[i,j] for k in range(N) for i in xrange(96) for j in xrange(96)) + \
                 min_curOnPeak*18.74 + min_curPeak*17.33 + min_curMidPeak*5.23
        
        for item in curPeak:
            model += min_curPeak >= item
        model += min_curPeak >= peak
        for item in curOnPeak:
            model += min_curOnPeak >= item
        model += min_curOnPeak >= onpeak
        for item in curMidPeak:
            model += min_curMidPeak >= item
        model += min_curOnPeak >= midpeak
        
    elif (tim.month in [5,6,7,8,9,10]) & (tim.isoweekday() in [6,7]):
        curPeak = [lpSum(lpSum(DayPower[eventid].tolist()[i]*X[i,j] for i in xrange(96)) + DailyPower['power'].tolist()[j]) for j in xrange(96)]
        
        model += lpSum(DayPower[eventid].tolist()[i]*X[i,j]*cost[i,j] for i in xrange(96) for j in xrange(96)) + min_curPeak*17.33
        
        for item in curPeak:
            model += min_curPeak >= item
        model += min_curPeak >= peak
        
    else:
        print "Error in month, day and time"
    
    # Create model constrains
    oldpower = DailyPower.loc[DailyPower[eventid] >= 0,[eventid,'ts']]
    newpower = DayPower.loc[DayPower[eventid] >= 0,[eventid]]
    power2 = newpower[eventid].tolist()

    positive_power = [p for p in power2 if p>0]
    M=len(positive_power)

    # Left and Right slots
    LS = oldpower['ts'][0]
    RS = oldpower['ts'][-1]

    # Set the precedence and the session duration constraints
    for i in xrange(LS):
        model += (lpSum(X[i,j] for j in range(96)) == 0)   
    # Down zeros
    for i in xrange(LS+M,96):
        model += (lpSum(X[i,j] for j in range(96)) == 0)    
    # Left zeros
    for i in xrange(LS,LS+M):
        model += (lpSum(X[i,j] for j in range(LS)) == 0)
    # Right zeros
    for i in xrange(LS,LS+M):
        model += (lpSum(X[i,j] for j in range(RS+1,96)) == 0)

    # Add the constraints
    for i in xrange(LS,LS+M):
        model += (lpSum(X[i,j] for j in range(LS,RS+1)) == 1)

    for i in xrange(LS,LS+M-1):
        model += (lpSum(X[i,j]*(j+1) for j in range(LS,RS+1)) == lpSum(X[i+1,j]*(j+1) for j in range(LS,RS+1))-1)
                
    model.solve()
    
    if model.status == 1:
        oldpower = DailyPower.loc[DailyPower[eventid] >= 0,[eventid,'ts']]
        newpower = DayPower.loc[DayPower[eventid] >= 0,[eventid]]
        power2 = newpower[eventid].tolist()

        positive_power = [p for p in power2 if p>0]
        M=len(positive_power)

        # Left and Right slots
        LS = oldpower['ts'][0]
        RS = oldpower['ts'][-1]

        slots = xrange(1,len(power2)+1)

        sol=[]
        for j in xrange(96):
            sol.append(sum(X[i,j].varValue*(j) for i in xrange(LS,LS+M)))
        corSlots=[slots[int(s)] for s in sol]
        ct=0
        power=[p*0 for p in power2]
        for s in sol:
            if s>0:
                power[int(s)]=positive_power[ct]
                ct=ct+1
            else:
                continue
        DailyPower.loc[DailyPower.index[LS:RS+1],eventid] = power[LS:RS+1]
        
        # calculate the new onpeak, peak and midpeak
        DailyPower['power'] = DailyPower['power'] + DailyPower[eventid].fillna(0)

        print('The solution'+' ' + eventid + ' '+'is optimal')
    else:
        print('The solution is suboptimal')
    return DailyPower


In [27]:
# run the optimization for each month's EV charging sessions

for month in range(6,7):
    # monthly date range
#     StartDate = ts2015[ts2015['month'] == month].day[0]
#     EndDate = ts2015[ts2015['month'] == month].day[-1]
    StartDate = '2016-6-1'
    EndDate = '2016-6-30'
    rng = map(lambda x: x.strftime('%Y-%m-%d'),pd.date_range(StartDate, EndDate, freq='D'))
    
    EndDate2 = (datetime.strptime(EndDate,'%Y-%m-%d')+timedelta(days=1)).strftime('%Y-%m-%d')
#     EVSE_Power = query_EVSE_eventids(StartDate,EndDate2)
    EVSE_Power = pd.read_csv('results/FlexDrivers/NewEVStatus/Opt5/'+'2015-'+str(month)+'Original.csv')
    EVSE_Power.index = EVSE_Power.ix[:,0].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    EVSE_Power.drop(EVSE_Power.columns[[0]], axis=1, inplace=True)
    
    # initialize the past peak demand values 
    DailyPower = []
    peak=0
    onpeak=0
    midpeak=0

    monthpeak = []
    monthonpeak = []
    monthmidpeak = []
    
    directory = 'results/FlexDrivers/NewEVStatus/Opt5/' + str(month)
    if not os.path.exists(directory):
        os.makedirs(directory)
    # run the daily optimization
    for day in rng:
        DailyPower = EVSE_Power[day]
        DailyPower = DailyPower.replace('',np.nan)
        DailyPower = DailyPower.dropna(axis=1,how='all')
        
        DriverOnList = DailyPower.columns
        N = len(DriverOnList)
        
        DailyPower['power'] = 0
        DailyPower['NonControlledPower'] = DailyPower.fillna(0).sum(axis=1)
        DailyPower['ts'] = range(96) # Add the tag of time step for each event id
        
        if DailyPower.empty:
            print 'EV Data on ' + day + ' is empty!'
            DailyPower.to_csv(directory + '/' + day +'.csv')
        else:
            for eventid in DriverOnList[::-1]:
                DailyPower = V2optimizeCost(DailyPower, eventid, onpeak, midpeak, peak)

            print 'Optimization on ' + day + ' is done successfully'
            DailyPower.to_csv(directory +'/'+ day +'.csv')
#     EVSE_Power[StartDate:EndDate].to_csv('results/FlexDrivers/NewEVStatus/Opt4/'+'2015-'+str(month)+'Original.csv')

The solution 44844165 is optimal
The solution 44852697 is optimal
The solution 44855983 is optimal
The solution 44858539 is optimal
The solution 44859267 is optimal
The solution 44864519 is optimal
The solution 44869173 is optimal
The solution 44871169 is optimal
The solution 44871479 is optimal
The solution 44874547 is optimal
The solution 44878989 is optimal
The solution 44882015 is optimal
The solution 44882039 is optimal
The solution 44882959 is optimal
The solution 44883463 is optimal
The solution 44885449 is optimal
The solution 44887003 is optimal
The solution 44887219 is optimal
Optimization on 2016-06-01 is done successfully
The solution 44878989 is optimal
The solution 44882039 is optimal
The solution 44882959 is optimal
The solution 44883463 is optimal
The solution 44885449 is optimal
The solution 44887003 is optimal
The solution 44887219 is optimal
The solution 44904773 is optimal
The solution 44908203 is optimal
The solution 44911421 is optimal
The solution 44915123 is opt

KeyboardInterrupt: 